In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
from config import client_id, client_secret
import re

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [5]:
def getURI(uri_list):
#     p = re.compile(r'(?:spotify:playlist:)(\w{22})')
    p = re.compile(r'(?:spotify:(?:album|playlist|artist|track):)(\w{22})')
    URIs = [p.match(item).group(1) for item in uri_list]
    return URIs

In [22]:
demo_list = ["spotify:playlist:37i9dQZF1DX76Wlfdnj7AP", "spotify:playlist:37i9dQZF1DX70RN3TfWWJh","spotify:playlist:37i9dQZF1DX35oM5SPECmN",
            "spotify:playlist:37i9dQZF1DX9BXb6GsGCLl","spotify:playlist:37i9dQZF1DWXx3Txis2L4x","spotify:playlist:37i9dQZF1DWV3VLITCZusq",
            "spotify:playlist:37i9dQZF1DX9oh43oAzkyx","spotify:playlist:37i9dQZF1DWTl4y3vgJOXW","spotify:playlist:37i9dQZF1DWUVpAXiEPK8P"]
len(demo_list)

9

In [57]:
def getPlaylistTracks(playlist_URI_list):
    # Tracks variable will hold the JSON dictionaries,
    # one dictionary for every playlist 
    
    clean_URIs = getURI(playlist_URI_list)
    json_dicts = [sp.playlist_items(uri) for uri in clean_URIs]
    
    tracks = []
    for i in range(len(json_dicts)):
        json = json_dicts[i]['items']
        for j in range(len(json)):
            tracks.append(json[j]['track']['id'])
    
    return tracks


In [60]:
track_ids = getPlaylistTracks(demo_list)
len(track_ids)

671

In [61]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [63]:
# loop over track ids 
tracks = []

for i in range(len(track_ids)):
  time.sleep(.5)
  track = getTrackFeatures(track_ids[i])
  tracks.append(track)

# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("workout.csv", sep = ',')

In [64]:
df.head()

,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,Goosebumps - Remix,Goosebumps (Remix),Travis Scott,2021-01-15,162802,85,0.841,0.41800,0.841,0.593,0.000000,0.1240,-7.846,0.0379,124.917,4
1,Paradise (feat. Dermot Kennedy),Paradise,MEDUZA,2020-10-30,167903,83,0.632,0.06890,0.632,0.595,0.000000,0.2090,-7.644,0.0401,124.114,4
2,INDUSTRY BABY (feat. Jack Harlow),INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,2021-07-23,212000,97,0.736,0.02030,0.736,0.704,0.000000,0.0501,-7.409,0.0615,149.995,4
3,Motley Crew,Motley Crew,Post Malone,2021-07-09,184213,84,0.797,0.09040,0.797,0.631,0.000004,0.0998,-3.818,0.0786,129.915,4
4,Levitating (feat. DaBaby),Levitating (feat. DaBaby),Dua Lipa,2020-10-01,203064,86,0.702,0.00883,0.702,0.825,0.000000,0.0674,-3.787,0.0601,102.977,4
